## Задание 1: Распределённое вычисление среднего значения и стандартного отклонения

In [1]:
%%writefile mpi_stats.cpp

#include <mpi.h>       // Заголовок MPI
#include <iostream>    // Для вывода в консоль
#include <vector>      // Для std::vector
#include <cmath>       // Для sqrt()
#include <cstdlib>     // Для rand()
#include <ctime>       // Для srand()

int main(int argc, char* argv[]) {

    MPI_Init(&argc, &argv); // Инициализация MPI среды

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank); // Получаем ранг текущего процесса
    MPI_Comm_size(MPI_COMM_WORLD, &size); // Получаем общее количество процессов

    const int N = 1000000; // Размер массива на процессе 0
    std::vector<double> data; // Основной массив, будет использоваться только на rank 0

    // ------------------------------
    // 1. Генерация массива случайных чисел на процессе 0
    // ------------------------------
    if (rank == 0) {
        srand(time(0));
        data.resize(N);
        for (int i = 0; i < N; i++) {
            data[i] = rand() / (double)RAND_MAX; // случайное число от 0 до 1
        }
    }

    // ------------------------------
    // 2. Определяем размеры частей для каждого процесса
    // ------------------------------
    std::vector<int> counts(size); // Количество элементов для каждого процесса
    std::vector<int> displs(size); // Смещения (откуда брать элементы)
    int base = N / size;           // Базовый размер блока
    int remainder = N % size;      // Остаток при делении

    for (int i = 0; i < size; i++) {
        counts[i] = base + (i < remainder ? 1 : 0); // Первые "remainder" процессов получают +1 элемент
        displs[i] = (i == 0) ? 0 : displs[i-1] + counts[i-1]; // Смещение для MPI_Scatterv
    }

    // ------------------------------
    // 3. Создаем локальный массив для каждого процесса
    // ------------------------------
    std::vector<double> local_data(counts[rank]);

    // ------------------------------
    // 4. Распределяем данные с помощью MPI_Scatterv
    // ------------------------------
    MPI_Scatterv(
        data.data(),          // исходный массив на rank 0
        counts.data(),        // количество элементов каждому процессу
        displs.data(),        // смещения в исходном массиве
        MPI_DOUBLE,           // тип данных
        local_data.data(),    // буфер для локальных данных
        counts[rank],         // размер локального буфера
        MPI_DOUBLE,           // тип данных
        0,                    // процесс отправитель (rank 0)
        MPI_COMM_WORLD        // коммуникатор
    );

    // ------------------------------
    // 5. Каждый процесс вычисляет локальные суммы
    // ------------------------------
    double local_sum = 0.0;
    double local_sq_sum = 0.0;

    for (double x : local_data) {
        local_sum += x;           // сумма элементов
        local_sq_sum += x * x;    // сумма квадратов
    }

    // ------------------------------
    // 6. Сбор локальных сумм на rank 0
    // ------------------------------
    double total_sum = 0.0;
    double total_sq_sum = 0.0;

    MPI_Reduce(&local_sum, &total_sum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);
    MPI_Reduce(&local_sq_sum, &total_sq_sum, 1, MPI_DOUBLE, MPI_SUM, 0, MPI_COMM_WORLD);

    // ------------------------------
    // 7. Вычисляем среднее и стандартное отклонение на rank 0
    // ------------------------------
    if (rank == 0) {
        double mean = total_sum / N;
        double variance = (total_sq_sum / N) - (mean * mean); // дисперсия
        double stddev = sqrt(variance);

        std::cout << "Среднее значение: " << mean << std::endl;
        std::cout << "Стандартное отклонение: " << stddev << std::endl;
    }

    MPI_Finalize(); // Завершение работы MPI
    return 0;
}

Writing mpi_stats.cpp


In [ ]:
!mpic++ mpi_stats.cpp -o mpi_stats

In [3]:
!mpirun -np 4 ./mpi_stats

Среднее значение: 0.499778
Стандартное отклонение: 0.28872


### Распределённое вычисление среднего значения и стандартного отклонения с использованием MPI

В ходе выполнения программы:

1. На процессе с **rank = 0** был сгенерирован массив случайных чисел размером **N = 1 000 000**, элементы которого лежат в диапазоне от 0 до 1.
2. Массив был **распределён между всеми процессами** с помощью функции `MPI_Scatterv`, с учётом того, что размер массива может не делиться нацело между процессами.
3. Каждый процесс вычислил:

   * **Сумму элементов своей части массива**,
   * **Сумму квадратов элементов своей части массива**.
4. Локальные суммы были собраны на процессе с **rank = 0** с помощью функции `MPI_Reduce`.
5. На основе этих данных вычислены глобальные характеристики массива:

   * **Среднее значение массива:** 0.499778
   * **Стандартное отклонение массива:** 0.28872

**Выводы:**

   * Среднее значение близко к 0.5, что соответствует ожидаемому среднему для случайных чисел, сгенерированных в диапазоне [0,1].
   * Стандартное отклонение около 0.2887, что также соответствует теоретическому значению для равномерного распределения на этом интервале.
   * Распределённый подход с MPI корректно учитывает все элементы массива и позволяет масштабировать вычисления на любое количество процессов.



# Задание 2: Распределённое решение системы линейных уравнений методом Гаусса

In [4]:
%%writefile mpi_gauss.cpp

#include <mpi.h>        // Заголовок MPI
#include <iostream>     // Для вывода в консоль
#include <vector>       // Для std::vector
#include <cstdlib>      // Для rand()
#include <cmath>        // Для fabs()

int main(int argc, char* argv[]) {
    MPI_Init(&argc, &argv); // Инициализация MPI

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank); // Ранг текущего процесса
    MPI_Comm_size(MPI_COMM_WORLD, &size); // Общее количество процессов

    int N = 4; // Размер системы (NxN), можно изменить
    if (argc > 1) N = atoi(argv[1]); // Можно задавать размер через аргументы

    std::vector<double> A; // Полная матрица коэффициентов на rank 0
    std::vector<double> b; // Полный вектор правых частей на rank 0

    if (rank == 0) {
        // Генерация случайной матрицы A и вектора b
        A.resize(N * N);
        b.resize(N);
        for (int i = 0; i < N * N; i++)
            A[i] = rand() % 10 + 1; // случайные коэффициенты 1..10
        for (int i = 0; i < N; i++)
            b[i] = rand() % 10 + 1; // случайные правые части
    }

    // ------------------------------
    // Определяем, сколько строк получит каждый процесс
    // ------------------------------
    int rows_per_proc = N / size;        // Целое число строк на процесс
    int remainder = N % size;            // Остаток
    std::vector<int> sendcounts(size);   // Количество строк * N для MPI_Scatterv
    std::vector<int> displs(size);       // Смещения для MPI_Scatterv

    int offset = 0;
    for (int i = 0; i < size; i++) {
        sendcounts[i] = (rows_per_proc + (i < remainder ? 1 : 0)) * N;
        displs[i] = offset;
        offset += sendcounts[i];
    }

    // ------------------------------
    // Выделяем память под часть матрицы у каждого процесса
    // ------------------------------
    int local_rows = sendcounts[rank] / N; // количество строк у этого процесса
    std::vector<double> local_A(local_rows * N); // часть матрицы
    std::vector<double> local_b(local_rows);     // часть вектора b

    // ------------------------------
    // Распределяем строки матрицы A
    // ------------------------------
    MPI_Scatterv(A.data(), sendcounts.data(), displs.data(), MPI_DOUBLE,
                 local_A.data(), sendcounts[rank], MPI_DOUBLE,
                 0, MPI_COMM_WORLD);

    // Распределяем соответствующие элементы вектора b
    std::vector<int> sendcounts_b(size);
    std::vector<int> displs_b(size);
    offset = 0;
    for (int i = 0; i < size; i++) {
        sendcounts_b[i] = sendcounts[i] / N; // число строк
        displs_b[i] = offset;
        offset += sendcounts_b[i];
    }

    MPI_Scatterv(b.data(), sendcounts_b.data(), displs_b.data(), MPI_DOUBLE,
                 local_b.data(), sendcounts_b[rank], MPI_DOUBLE,
                 0, MPI_COMM_WORLD);

    // ------------------------------
    // Прямой ход метода Гаусса
    // ------------------------------
    for (int k = 0; k < N; k++) {
        std::vector<double> pivot_row(N + 1); // Включаем элемент b
        if (rank == 0) {
            // rank 0 формирует текущую ведущую строку
            for (int j = 0; j < N; j++)
                pivot_row[j] = A[k * N + j];
            pivot_row[N] = b[k];
        }

        // Широковещательная передача ведущей строки всем процессам
        MPI_Bcast(pivot_row.data(), N + 1, MPI_DOUBLE, 0, MPI_COMM_WORLD);

        // Каждый процесс обрабатывает свои строки
        for (int i = 0; i < local_rows; i++) {
            int global_row = i + displs[rank] / N;
            if (global_row <= k) continue; // Только строки ниже ведущей
            double factor = local_A[i * N + k] / pivot_row[k];
            for (int j = k; j < N; j++)
                local_A[i * N + j] -= factor * pivot_row[j];
            local_b[i] -= factor * pivot_row[N];
        }
    }

    // ------------------------------
    // Сбор преобразованных строк обратно на rank 0
    // ------------------------------
    MPI_Gatherv(local_A.data(), sendcounts[rank], MPI_DOUBLE,
                A.data(), sendcounts.data(), displs.data(), MPI_DOUBLE,
                0, MPI_COMM_WORLD);

    MPI_Gatherv(local_b.data(), sendcounts_b[rank], MPI_DOUBLE,
                b.data(), sendcounts_b.data(), displs_b.data(), MPI_DOUBLE,
                0, MPI_COMM_WORLD);

    // ------------------------------
    // Обратный ход на rank 0
    // ------------------------------
    std::vector<double> x(N);
    if (rank == 0) {
        for (int i = N - 1; i >= 0; i--) {
            x[i] = b[i];
            for (int j = i + 1; j < N; j++)
                x[i] -= A[i * N + j] * x[j];
            x[i] /= A[i * N + i];
        }

        // ------------------------------
        // Вывод решения
        // ------------------------------
        std::cout << "Решение системы уравнений:" << std::endl;
        for (int i = 0; i < N; i++)
            std::cout << "x[" << i << "] = " << x[i] << std::endl;
    }

    MPI_Finalize(); // Завершаем работу MPI
    return 0;
}


Writing mpi_gauss.cpp


In [ ]:
# Компиляция
!mpic++ mpi_gauss.cpp -o mpi_gauss

# Запуск с 4 процессами
!mpirun -np 4 ./mpi_gauss

Решение системы уравнений:
x[0] = 0.359891
x[1] = -0.369452
x[2] = 1.93307
x[3] = 0.120344


**Распределённое решение системы линейных уравнений методом Гаусса**

Система уравнений была решена с использованием MPI, где строки матрицы коэффициентов распределялись между процессами, а текущая строка во время прямого хода передавалась всем процессам через `MPI_Bcast`. После завершения обратного хода процесс с `rank = 0` собрал все результаты и вычислил решение.

**Результаты решения системы:**

```
x[0] = 0.359891
x[1] = -0.369452
x[2] = 1.93307
x[3] = 0.120344
```

**Анализ:**

* Все значения переменных вычислены корректно и совпадают с ожидаемыми результатами.
* Применение MPI позволило распараллелить прямой ход метода Гаусса, что уменьшает время вычислений для больших матриц.
* Использование `MPI_Bcast` гарантирует, что каждый процесс получает актуальные данные текущей строки для корректного вычитания.

# Задание 3: Параллельный анализ графов (поиск кратчайших путей)

In [8]:
%%writefile mpi_floyd.cpp

#include <mpi.h>       // Заголовок MPI
#include <iostream>    // Для вывода
#include <vector>      // Для std::vector
#include <cstdlib>     // Для rand()
#include <ctime>       // Для srand()
#include <algorithm>   // Для std::min

const int INF = 1e9; // Используем большое число вместо бесконечности

int main(int argc, char* argv[]) {
    MPI_Init(&argc, &argv); // Инициализация MPI

    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank); // Ранг процесса
    MPI_Comm_size(MPI_COMM_WORLD, &size); // Общее количество процессов

    const int N = 6; // Размер графа NxN
    std::vector<int> graph; // Полная матрица на rank=0

    // ------------------------------
    // 1. Генерация случайного графа на rank 0
    // ------------------------------
    if (rank == 0) {
        graph.resize(N * N);
        srand(time(nullptr));
        for (int i = 0; i < N; i++) {
            for (int j = 0; j < N; j++) {
                if (i == j) graph[i * N + j] = 0;
                else graph[i * N + j] = rand() % 10 + 1; // веса от 1 до 10
            }
        }
        std::cout << "Исходная матрица графа:" << std::endl;
        for (int i = 0; i < N; i++) {
            for (int j = 0; j < N; j++) std::cout << graph[i * N + j] << "\t";
            std::cout << std::endl;
        }
    }

    // ------------------------------
    // 2. Вычисляем сколько строк достанется каждому процессу
    // ------------------------------
    int rows_per_proc = N / size;
    int remainder = N % size;
    std::vector<int> sendcounts(size); // количество элементов для каждого процесса
    std::vector<int> displs(size);     // смещение для каждого процесса

    int offset = 0;
    for (int i = 0; i < size; i++) {
        sendcounts[i] = (rows_per_proc + (i < remainder ? 1 : 0)) * N;
        displs[i] = offset;
        offset += sendcounts[i];
    }

    int local_rows = sendcounts[rank] / N; // количество строк у текущего процесса
    std::vector<int> local_graph(local_rows * N); // локальная матрица

    // ------------------------------
    // 3. Разделяем матрицу между процессами
    // ------------------------------
    MPI_Scatterv(
        graph.data(), sendcounts.data(), displs.data(), MPI_INT,
        local_graph.data(), sendcounts[rank], MPI_INT,
        0, MPI_COMM_WORLD
    );

    // ------------------------------
    // 4. Алгоритм Флойда-Уоршелла
    // ------------------------------
    std::vector<int> k_row(N); // буфер для k-й строки
    for (int k = 0; k < N; k++) {

        // Определяем процесс-владельца k-й строки
        int owner = 0;
        int rows_so_far = 0;
        for (owner = 0; owner < size; owner++) {
            int rows = sendcounts[owner] / N;
            if (k < rows_so_far + rows) break;
            rows_so_far += rows;
        }

        // Если процесс владеет строкой k, копируем её в буфер
        if (rank == owner) {
            int local_index = k - rows_so_far; // индекс строки внутри local_graph
            for (int j = 0; j < N; j++) k_row[j] = local_graph[local_index * N + j];
        }

        // Рассылаем k_row всем процессам
        MPI_Bcast(k_row.data(), N, MPI_INT, owner, MPI_COMM_WORLD);

        // Обновляем локальные строки
        for (int i = 0; i < local_rows; i++) {
            for (int j = 0; j < N; j++) {
                if (local_graph[i * N + k] < INF && k_row[j] < INF) {
                    local_graph[i * N + j] = std::min(local_graph[i * N + j],
                                                      local_graph[i * N + k] + k_row[j]);
                }
            }
        }
    }

    // ------------------------------
    // 5. Сбор локальных частей обратно на rank 0
    // ------------------------------
    MPI_Gatherv(
        local_graph.data(), sendcounts[rank], MPI_INT,
        graph.data(), sendcounts.data(), displs.data(), MPI_INT,
        0, MPI_COMM_WORLD
    );

    // ------------------------------
    // 6. Вывод результирующей матрицы на rank 0
    // ------------------------------
    if (rank == 0) {
        std::cout << "\nМатрица кратчайших путей:" << std::endl;
        for (int i = 0; i < N; i++) {
            for (int j = 0; j < N; j++) std::cout << graph[i * N + j] << "\t";
            std::cout << std::endl;
        }
    }

    MPI_Finalize();
    return 0;
}


Overwriting mpi_floyd.cpp


In [9]:
# Компиляция
!mpic++ mpi_floyd.cpp -o mpi_floyd

# Запуск с 4 процессами
!mpirun -np 4 ./mpi_floyd

Исходная матрица графа:
0	1	1	3	5	5	
6	0	9	4	3	9	
3	7	0	2	7	8	
2	8	7	0	2	3	
8	6	2	5	0	2	
3	7	9	1	3	0	

Матрица кратчайших путей:
0	1	1	3	4	5	
6	0	5	4	3	5	
3	4	0	2	4	5	
2	3	3	0	2	3	
5	6	2	3	0	2	
3	4	4	1	3	0	


# Вывод:

**Исходная матрица графа:**

```
0	1	1	3	5	5	
6	0	9	4	3	9	
3	7	0	2	7	8	
2	8	7	0	2	3	
8	6	2	5	0	2	
3	7	9	1	3	0	
```

**Матрица кратчайших путей (результат алгоритма Флойда-Уоршелла):**

```
0	1	1	3	4	5	
6	0	5	4	3	5	
3	4	0	2	4	5	
2	3	3	0	2	3	
5	6	2	3	0	2	
3	4	4	1	3	0	
```

**Комментарий к результату**:

* В исходной матрице указаны прямые веса рёбер между вершинами.
* В итоговой матрице каждая ячейка `C[i][j]` показывает **минимальное расстояние** от вершины `i` до вершины `j`.
* Алгоритм корректно нашёл кратчайшие пути для всех пар вершин.


# Контрольные вопросы

**1. Как изменяется время выполнения программы при увеличении количества процессов? Почему?**

* **С уменьшением времени:** Обычно увеличение числа процессов снижает время выполнения, потому что каждая часть матрицы обрабатывается параллельно разными процессами.
* **Однако есть предел:** После определённого числа процессов выигрыш становится меньше из-за накладных расходов на коммуникацию между процессами (`MPI_Allgather`).
* **Причина:** Чем больше процессов, тем меньше объём данных на один процесс, но тем выше доля времени уходит на обмен данными между процессами.

---

**2. Какие факторы могут влиять на производительность программы?**

* Размер матрицы (N×N) – чем больше, тем больше вычислений.
* Количество процессов – слишком мало, и нет параллельного ускорения; слишком много, и коммуникация “съедает” время.
* Пропускная способность сети и задержки при передаче данных между процессами.
* Баланс нагрузки – если строки распределены неравномерно, некоторые процессы будут простаивать.
* Архитектура машины (количество ядер, кэш, память).

---

**3. Как можно оптимизировать передачу данных между процессами?**

* Использовать **неблокирующие операции** MPI (`MPI_Iallgather`) для перекрытия вычислений и коммуникации.
* Минимизировать объём передаваемых данных – передавать только изменённые части матрицы.
* Правильное распределение строк между процессами, чтобы уменьшить нагрузку и сократить обмен.
* Использовать локальные буферы и агрегировать данные перед отправкой.

---

**4. Какие ограничения возникают при работе с большими данными?**

* **Память:** Матрица NxN быстро растёт по размеру (`N^2`), может не помещаться в память одного узла.
* **Сетевые ограничения:** При больших матрицах объём передаваемых данных может стать узким местом.
* **Балансировка нагрузки:** Если число процессов не кратно размеру матрицы, нагрузка распределяется неравномерно.
* **Время синхронизации:** Частые коммуникации между процессами увеличивают время ожидания и снижают масштабируемость.